# 1.使用请求/响应模式发送更透明的消息

## 以下代码是模拟互联车辆的门锁

In [ ]:
from paho.mqtt.packettypes import PacketTypes
import ssl
import time
from paho.mqtt.properties import Properties
import paho.mqtt.client as mqtt
import logging
import json

logging.basicConfig(level=logging.DEBUG)
certificates_path = 'certificates'
endpoint = 'a2u3inau7j0faa-ats.iot.ap-northeast-1.amazonaws.com'
# endpoint=  'a2jtec7plm36gl.ats.iot.cn-north-1.amazonaws.com.cn'

certs = {
    "cafile": certificates_path+"/AmazonRootCA1.pem",
    "certfile": certificates_path+"/car-certificate.pem.crt",
    "keyfile": certificates_path+"/car-private.pem.key",
}

def on_connect(mqttc, userdata, flags, reasonCode, properties=None):
    mqttc.subscribe('home07/main_door/lock', qos=0)

def on_message(mqttc, userdata, message):
    raw_payload = str(message.payload.decode("utf-8"))
    logging.debug(f"Received a message on topic: '{message.topic}', payload: '{raw_payload}'")
    logging.debug(f"Main door LOCK request with parameters: '{str(message.properties.CorrelationData)}'")
    properties = Properties(PacketTypes.PUBLISH)
    properties.CorrelationData = message.properties.CorrelationData
    response_topic = message.properties.ResponseTopic
    payload = "USER_IS_NOT_AUTHENTICATED"
    mqttc.publish(response_topic, payload, qos=0, properties=properties)

mqttc = mqtt.Client("Car", protocol=mqtt.MQTTv5)
logger = logging.getLogger(__name__)
mqttc.enable_logger(logger)

mqttc.on_message = on_message
mqttc.on_connect = on_connect

mqttc.tls_set(certs["cafile"],
              certfile=certs["certfile"],
              keyfile=certs["keyfile"],
              cert_reqs=ssl.CERT_REQUIRED,
              tls_version=ssl.PROTOCOL_TLSv1_2,
              ciphers=None)

mqttc.connect(endpoint, 8883)
mqttc.loop_forever()

# 4.使用消息过期、会话过期和Clean Start功能更好地控制设备行为

## 以下代码是模拟互联车辆的车门和空调

In [ ]:
from paho.mqtt.packettypes import PacketTypes
import ssl
import time
from paho.mqtt.properties import Properties
import paho.mqtt.client as mqtt
import logging
import json
logging.basicConfig(level=logging.DEBUG)
session_expiry_interval = 300
certificates_path = 'certificates'
#endpoint = 'a2u3inau7j0faa-ats.iot.ap-northeast-1.amazonaws.com'
endpoint=  'a2jtec7plm36gl.ats.iot.cn-north-1.amazonaws.com.cn'

certs = {
    "cafile": certificates_path+"/AmazonRootCA1.pem",
    "certfile": certificates_path+"/car-certificate.pem.crt",
    "keyfile": certificates_path+"/car-private.pem.key",
}



def sub_client_on_connect(mqtt_sub, user_data, flags, reason_code, properties=None):
    mqtt_sub.subscribe('vehicle/#', qos=1)
    time.sleep(1)

def sub_client_on_message(mqtt_sub, user_data, message):
    raw_payload = str(message.payload.decode("utf-8"))
    logging.debug(f"Received a message on topic: '{message.topic}', payload: '{raw_payload}'")


# Subscriber client
mqtt_sub = mqtt.Client("Car", protocol=mqtt.MQTTv5)
logger = logging.getLogger(__name__)
mqtt_sub.enable_logger(logger)
    
mqtt_sub.on_message = sub_client_on_message
mqtt_sub.on_connect = sub_client_on_connect

mqtt_sub.tls_set(certs["cafile"],
                   certfile=certs["certfile"],
                   keyfile=certs["keyfile"],
                   cert_reqs=ssl.CERT_REQUIRED,
                   tls_version=ssl.PROTOCOL_TLSv1_2,
                   ciphers=None)

# Connect to sub client
properties_connect = Properties(PacketTypes.CONNECT)
properties_connect.SessionExpiryInterval = session_expiry_interval
mqtt_sub.connect(endpoint, 8883, clean_start=True, properties=properties_connect)

mqtt_sub.loop_forever()